In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from models import *

In [54]:
# Find top 5 locations for a given location of a user

subquery = (Locations
            .select(Locations.street.alias('street'), fn.sqrt(fn.pow(51.69-Locations.dis_lat, 2) + 
                                                              fn.pow(5.30-Locations.dis_long, 2))
                    .alias('dist'), Locations.dis_long, Locations.dis_lat)
            .order_by(SQL('dist'))
            .limit(5)
            .alias('subquery'))

# pick one image for each of the top 5 locations

images = (Images
          .select(fn.min(Images.id).alias('id'), Images.street, Images.url, Images.title,
                  subquery.c.dis_long, subquery.c.dis_lat)
          .join(subquery, on =
                (Images.street == subquery.c.street))
          .group_by(Images.street)
          .dicts()
          .execute())

pd.DataFrame(images)

,dis_lat,dis_long,id,street,title,url
0,51.68973,5.30022,94,Korenbrugstraat,Binnendieze met beeld op hoek Korenbrugstraat/...,http://denbosch.hosting.deventit.net/HttpHandl...
1,51.69034,5.30036,108,Visstraat,Straatbeeld Visstraat te Den Bosch begin jaren...,http://denbosch.hosting.deventit.net/HttpHandl...
2,51.68998,5.29982,430,Lepelstraat,Een schippersorkest loopt de Korenbrugstraat i...,http://denbosch.hosting.deventit.net/HttpHandl...
3,51.68961,5.29943,745,Sint Jansstraat,"Zijgevel van het Keershuis, Lepelstraat 12.",http://denbosch.hosting.deventit.net/HttpHandl...
4,51.68998,5.29982,6411,Vismarkt,Markttafereel.,http://denbosch.hosting.deventit.net/HttpHandl...
